In [3]:
from crewai import LLM
import os

llm=LLM(
    model='deepseek-r1-distill-llama-70b',
    temperature=0.1,
    api_key=os.getenv("GROQ_API_KEY")
)

In [4]:
from crewai import Agent,Task,Crew
from crewai_tools import FileReadTool


file_reader=FileReadTool('./data/customers.csv')


class DataValidatorTool:
    def run(self,dataframe):
        report=[]
        for file_name,df in dataframe.items():
            summary={
                "file":file_name,
                "rows":len(df),
                "columns":len(df.columns),
                "null_count":int(df.isnull().sum().sum()),
                "duplicate_count":int(df.duplicated().sum())
            }
            
            report.append(summary)
        return report


validator_tool=DataValidatorTool()    
            

loader_agent=Agent(
    role="Data Loader",
    goal="Read local CSV/JSON files prepare for Validation",
    backstory=""" This agent can scan the local directory ,finds the CSV/JSON files 
                   and read them into dataframe, and collects metadata for validation""",
    tools=[file_reader],
    verbose=True,
    llm=llm 
)

validator_agent=Agent(
    role ="Data validator",
    goal="Check dataset for missing values,duplicates, and Anomolies",
    backstory="""This Agent recieves Dataframes from the loader and runs qulaity checks so that 
    the clean ,reliable data can be stored or processe further.""",
    tool=[validator_tool],
    llm=llm,
    verbose=True
)

validate_task=Task(
    description=(
        "Run data qualit,y check on provided Dataframes.\n"
        "1.Count total number of rows"
        "2.Count Missing values (NAN).\n"
        "3.Count duplicate Rows \n"
        "Return Summary report for each file"
    ),
    agent=validator_agent,
    expected_output="A JSON report containing file name, rows, columns, null_count, and duplicate_count for each dataset."
)



load_task=Task(
    description=(
        "1.Scan the 'data/' folder for .csv.\n"
        "2.For each file,read it contents into a Pandas Dataframe.\n"
        "3.Collect metadata: file name,size,number of rows and columns\n"
        "4.Pass DataFrames to the next agent for validation"
        ),
    agent=loader_agent,
    expected_output="A dictionary of file names mapped to Pandas DataFrames for validation."
    
)

crew=Crew(
    agents=[loader_agent,validator_agent],
    tasks=[load_task,validate_task],
    verbose=True
)

res=crew.kickoff()
print("\n=== Data Validation Report ===")
print(res)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d38fdcb1-4b18-4814-9c13-3020e6ff4ebf                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Loader                                                                                             │
│                                                                                                                 │
│  Task: 1.Scan the 'data/' folder for .csv.                                                                      │
│  2.For each file,read it contents into a Pandas Dataframe.                                                      │
│  3.Collect metadata: file name,size,number of rows and columns                                                  │
│  4.Pass DataFrames to the next agent for validation                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call.    │
│  You passed model=deepseek-r1-distill-llama-70b                                                                 │
│   Pass model as E.g. For 'Huggingface' inference endpoints pass in                                              │
│  `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: b0475962-ceb1-4d42-944a-f72b4942f89c                                                                     │
│  Agent: Data Loader                                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: d38fdcb1-4b18-4814-9c13-3020e6ff4ebf                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=deepseek-r1-distill-llama-70b
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers